In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [35]:
br_stocks = pd.read_csv("acoes_brasil.csv", sep=";", decimal=",", thousands='.').fillna(0)
print(br_stocks.columns)
# print(br_stocks.dtypes)
br_stocks.head()

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')


,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,...,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,AALR3,10.41,0.00,-4.30,1.42,0.49,30.29,-1.09,-25.30,-99.75,...,0.34,0.65,0.45,0.13,0.00,1.898619e+06,7.32,-2.42,-0.01,1.231428e+09
1,ABCB4,19.86,6.07,5.71,0.83,0.08,31.62,25.25,19.35,4.38,...,0.10,0.90,0.07,18.70,12.95,1.226095e+07,23.98,3.48,0.39,4.490150e+09
2,ABEV3,12.98,5.86,14.34,2.41,1.54,49.95,21.50,17.26,11.52,...,0.64,0.35,0.62,10.72,14.46,3.329247e+08,5.40,0.91,2.64,2.047998e+11
3,ADHM3,1.56,0.00,-4.09,-1.29,105.12,100.00,-13548.48,-18860.61,-5.69,...,-81.72,82.72,0.14,0.00,0.00,0.000000e+00,-1.21,-0.38,-0.58,2.543977e+07
4,AERI3,0.81,0.00,-5.82,0.74,0.20,13.03,9.37,-3.59,2.23,...,0.27,0.73,0.93,0.00,0.00,6.355384e+06,1.10,-0.14,0.00,6.206329e+08


In [41]:
# br_stocks.apply(lambda x : x[ x["P/L"] > 3 ].sort_values(by="P/L").head(20))
top_equity_df = br_stocks[ (br_stocks["P/L"] > 3) & (br_stocks["P/L"] < 20) ].sort_values(by="P/L").head(50)
top_equity_tickers = set(top_equity_df["TICKER"].unique())
print(top_equity_tickers)
# print(top_equity_df)
br_stocks["point_1"] = br_stocks.apply(lambda x : 2 if x["TICKER"] in top_equity_tickers else 0, axis=1)

top_dy_df = br_stocks[ (br_stocks["DY"] > 5) & (br_stocks["DY"] < 20) ]
top_dy_tickers = set(top_dy_df["TICKER"].unique())
print(top_dy_tickers)
# print(top_dy_df)
br_stocks["point_2"] = br_stocks.apply(lambda x : 0.5 if x["TICKER"] in top_dy_tickers else 0, axis=1)

top_pvp_df = br_stocks[ (br_stocks["P/VP"] > 0) & (br_stocks["P/VP"] < 2) ]
top_pvp_tickers = set(top_pvp_df["TICKER"].unique())
print(top_pvp_tickers)
# print(top_pvp_df)
br_stocks["point_3"] = br_stocks.apply(lambda x : 1 if x["TICKER"] in top_pvp_tickers else 0, axis=1)

top_net_margin_df = br_stocks[ (br_stocks["MARG. LIQUIDA"] > 10) & (br_stocks["MARG. LIQUIDA"] < 100) ]
top_net_margin_tickers = set(top_net_margin_df["TICKER"].unique())
print(top_net_margin_tickers)
# print(top_net_margin_df)
br_stocks["point_4"] = br_stocks.apply(lambda x : 1 if x["TICKER"] in top_net_margin_tickers else 0, axis=1)

top_gross_margin_df = br_stocks[ (br_stocks["MARGEM BRUTA"] > 10) & (br_stocks["MARGEM BRUTA"] < 100) ]
top_gross_margin_tickers = set(top_gross_margin_df["TICKER"].unique())
print(top_gross_margin_tickers)
# print(top_gross_margin_df)
br_stocks["point_4"] = br_stocks.apply(lambda x : 1 if x["TICKER"] in top_gross_margin_tickers else 0, axis=1)

br_stocks["point_sum"] = sum([br_stocks["point_%d" % i] for i in range(1, 5)])

{'FIGE3', 'HBTS5', 'GOAU4', 'MEAL3', 'EUCA4', 'VALE3', 'CTKA3', 'GOAU3', 'NEOE3', 'POSI3', 'EPAR3', 'SAPR11', 'JALL3', 'CIEL3', 'CTKA4', 'NAFG4', 'COCE5', 'CMIG4', 'G2DI33', 'BNBR3', 'CLSC4', 'BAZA3', 'GGBR4', 'TECN3', 'CGRA3', 'ROMI3', 'MTRE3', 'CLSC3', 'PETR3', 'EEEL4', 'BMKS3', 'CEDO3', 'PLAS3', 'AURE3', 'SAPR3', 'SUZB3', 'SAPR4', 'CAMB4', 'CEDO4', 'HAGA3', 'CAMB3', 'GGBR3', 'ALSO3', 'PETR4', 'TIET11', 'EUCA3', 'MTSA4', 'BBAS3', 'EEEL3', 'BRAP4'}
{'RAPT3', 'GOAU4', 'USIM5', 'MRFG3', 'EVEN3', 'GOAU3', 'RPAD5', 'LREN3', 'POMO4', 'EQPA3', 'CMIG4', 'CGAS3', 'EQPA5', 'EGIE3', 'CGAS5', 'ENBR3', 'KLBN4', 'UCAS3', 'LAND3', 'BGIP3', 'WIZC3', 'BMKS3', 'CSRN3', 'CPLE11', 'BRSR5', 'SAPR4', 'CMIN3', 'BEES3', 'FESA4', 'TAEE11', 'JBSS3', 'EKTR4', 'REDE3', 'SANB3', 'TAEE3', 'TASA3', 'CXSE3', 'BRAP4', 'ENMT3', 'HBTS5', 'BRAP3', 'SOND5', 'ALUP4', 'MELK3', 'BRBI11', 'CSRN6', 'EMAE3', 'GEPA4', 'CEBR5', 'CEBR3', 'TAEE4', 'CPLE6', 'BGIP4', 'VIVT3', 'ABEV3', 'CPLE3', 'FESA3', 'TASA4', 'TKNO4', 'CSRN5', 'K

In [42]:
br_stocks.sort_values(by="point_sum")

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,...,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO,point_1,point_2,point_3,point_4,point_sum
0,AALR3,10.41,0.00,-4.30,1.42,0.49,30.29,-1.09,-25.30,-99.75,...,1.898619e+06,7.32,-2.42,-0.01,1.231428e+09,0,0.0,1,1,2.0
1,ABCB4,19.86,6.07,5.71,0.83,0.08,31.62,25.25,19.35,4.38,...,1.226095e+07,23.98,3.48,0.39,4.490150e+09,0,0.5,1,1,2.5
2,ABEV3,12.98,5.86,14.34,2.41,1.54,49.95,21.50,17.26,11.52,...,3.329247e+08,5.40,0.91,2.64,2.047998e+11,0,0.5,0,1,1.5
3,ADHM3,1.56,0.00,-4.09,-1.29,105.12,100.00,-13548.48,-18860.61,-5.69,...,0.000000e+00,-1.21,-0.38,-0.58,2.543977e+07,0,0.0,0,0,0.0
4,AERI3,0.81,0.00,-5.82,0.74,0.20,13.03,9.37,-3.59,2.23,...,6.355384e+06,1.10,-0.14,0.00,6.206329e+08,0,0.0,1,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,WIZC3,5.07,8.75,7.75,1.96,0.34,58.20,36.98,10.49,2.20,...,5.137681e+06,2.59,0.66,-0.47,8.139281e+08,0,0.5,1,1,2.5
616,WLMM3,27.00,2.92,8.67,1.50,1.04,12.56,7.05,5.11,6.29,...,7.650290e+03,17.94,3.11,0.72,1.227667e+09,0,0.0,1,1,2.0
617,WLMM4,39.32,2.21,12.63,2.19,1.52,12.56,7.05,5.11,9.15,...,9.257545e+04,17.94,3.11,1.04,1.227667e+09,0,0.0,0,1,1.0
618,YDUQ3,19.62,0.00,54.99,1.93,0.62,58.53,16.35,2.27,7.63,...,1.100053e+08,10.16,0.36,0.06,6.058141e+09,0,0.0,1,1,2.0
